# Transform mask into COSTO .json

In [229]:
import re
from os import listdir, PathLike
from os.path import join
from pipeline.image_handeling.data_utility import load_stack
from skimage.measure import find_contours, regionprops_table
from numpyencoder import NumpyEncoder
import json

def mask_to_json(mask_fold_path: PathLike, save_path: PathLike, channel:str)->None:
  """Function that converts .tif labeled masks from np.array stacks into COCO .json format files.
  Saves the file into the given location.
  Args:
      mask_fold_path (PathLike): Folder of the masks.
      save_path (PathLike): Path where the .json is going to be saved. Must include the filename and the ending ".json".
      channel (string): The channel that should be transformed into .json.
  Returns:
      None, saves the .json into the given location."""
  img_path = []
  for file in sorted(listdir(mask_fold_path)):
      if channel in file:
        img_path.append(join(mask_fold_path,file))
  im_num = re.findall('f\d+', str(img_path))
  max_number = max(int(x[1:]) for x in im_num)
  max_number

  img_stack = load_stack(img_paths=img_path, channels=channel, frame_range=range(max_number), return_2D=True)

  annotations_lst = []
  images_lst = []
  for frame, img in enumerate(img_stack):
    outline_region =regionprops_table(img, properties=('label','bbox','area'))
    frame_name = img_path[frame].rsplit('/',1)[1]
    labels = outline_region['label']
    bbox0 = outline_region['bbox-0']
    bbox1 = outline_region['bbox-1']
    bbox2 = outline_region['bbox-2']
    bbox3 = outline_region['bbox-3']
    area = outline_region['area']

    for i in range(labels.shape[0]):
        label = labels[i]
        segmentation_lst=[]
        regionmask=(img==label)
        coords = find_contours(regionmask)
        for y,x in coords[0]:
            segmentation_lst.append(x+0.5)
            segmentation_lst.append(y+0.5)
        
        annotations_lst.append({'id':label, 
                                'image_id':frame+1,
                                'category_id':1,
                                'segmentation':[segmentation_lst],
                                'area':area[i],
                                'bbox':[bbox0[i], bbox1[i], bbox2[i], bbox3[i]],
                                'iscrowd': 0,
                                'attributes': {'occluded': False}})

    images_lst.append({'id': frame+1,
              'width': img.shape[1],
              'height': img.shape[0],
              'file_name': frame_name,
              'license': 0,
              'flickr_url': '',
              'coco_url': '',
              'date_captured': 0})

  json_dict = {'licenses': [{'name': '', 'id': 0, 'url': ''}],
          'info': {'contributor': '',
            'date_created': '',
            'description': '',
            'url': '',
            'version': '',
            'year': ''},
          'categories': [{'id': 1, 'name': 'Cell', 'supercategory': ''}],
          'images': images_lst,
          'annotations':annotations_lst}

  with open(save_path, "w") as file:
      json.dump(json_dict, file, cls=NumpyEncoder)

In [230]:

mask_folder = '/home/Fabian/ImageData/TrackingTestFiles/NeutrophilTrackingTest/mfap4-mpx_isohypo_2h_WT-MaxIP_s1/Masks_Cellpose'
channel = 'GFP'
savefile = '/home/Fabian/ImageData/Seriestest.json'

mask_to_json(mask_fold_path=mask_folder, save_path=savefile, channel=channel)

[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)]


# Reconstruct mask based on COSTO .json

In [231]:
import json
import numpy as np
from skimage.draw import polygon2mask
from os import PathLike

def mask_from_json(path:PathLike)->tuple[list[str],np.array]:
    """Recreates an np.array like stack of masks based on an COCO .json file.
    Return the np.array stack of masks and a list fo the Mask names.
    Args:
        path (PathLike): Path of the .json file to be loaded.
    Returns:
        tuple[list[str],np.array]: List of the image names, np.array containing the loaded masks"""
    file = open(path)
    json_file = json.load(file)

    mask_lst = []
    name_lst = []
    width = json_file['images'][0]['width']
    height = json_file['images'][0]['height']
    annotations = json_file['annotations']
    n_frame = len(json_file['images'])
    mask = np.zeros((n_frame,width,height), dtype='int')
    for images in json_file['images']:
        name_lst.append(images['file_name'])
    for annot in annotations:
        segmentation = annot['segmentation'][0]
        id = int(annot['id'])
        frame_id = int(annot['image_id'])
        coord_lst = []
        for point in range(0, len(segmentation),2):
            coord_lst.append((round(segmentation[point+1]), round(segmentation[point])))
        tempmask =  polygon2mask((width,height),coord_lst)
        tempmask[tempmask!=0] = id
        mask[frame_id-1] = mask[frame_id-1] + tempmask
    mask_lst.append(mask)
    mask_stack = np.squeeze(np.stack(mask_lst))
    return name_lst, mask_stack

In [232]:
path = '/home/Fabian/ImageData/instances_Series.json'
name_lst, mask_stack = mask_from_json(path)

# Play area

In [9]:
import numpy as np

In [24]:
polygons = np.array([(23,25),(12,15)], np.int32)

In [28]:
polygons = np.array([], np.int32)

In [25]:
polygons.size

4

In [29]:
if not polygons.size > 0:
    print('empty')

empty


In [13]:
polygons

array([], dtype=int32)

In [1]:
import cv2
from tifffile import imread

In [2]:
img_path = '/home/Fabian/ImageData/240502-minoSOG_mito_L10/HEKA_c1031_c1829_miniSOG_80%_435_2min_40min_002_Merged_s1/Images_Registered/YFP_s01_f0001_z0001.tif'

In [3]:
img = imread(img_path)

In [30]:
cv2.namedWindow("Draw ROI of the Wound", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Draw ROI of the Wound", 1024, 1024)
cv2.imshow("Draw ROI of the Wound",img)
f=0
while f!=-1:
    key = cv2.waitKeyEx(1) & 0xFF
    if key != 255:
        print(f'{key=}')
    if key == ord("q"):
        f = -1
cv2.destroyAllWindows()

key=119
key=100
key=115
key=97
key=83
key=82
key=81
key=84
key=108
key=107
key=106
key=105
key=107
key=108
key=108
key=107
key=106
key=105
key=107
key=108
key=107
key=106
key=105
key=107
key=103
key=103
key=102
key=114
key=101
key=100
key=103
key=113
